In [9]:
import json
import yaml
import psycopg2
import requests
import os

In [5]:
def make_db_connection_psycopg2(database: str, autocommit: bool = False):

    #assert database in list(conn_dict.keys()), "server couldn't be recognized in the config file"

    
    database="blog"
    host="localhost"
    port="5432"
    user="postgres"
    password="matin9862"
    
    conn = psycopg2.connect(user=user, password=password, host=host, dbname=database)
    cursor = conn.cursor()

    if autocommit:
        conn.autocommit = True

    return conn, cursor


In [6]:
conn, curs = make_db_connection_psycopg2(database="blog")

In [7]:
conn

<connection object at 0x7fa41627df80; dsn: 'user=postgres password=xxx dbname=blog host=localhost', closed: 0>

In [8]:
conn, cursor = make_db_connection_psycopg2("blog")
# query = """
# SELECT * FROM information_schema.tables WHERE table_schema = 'public';
# "

query = """
SELECT * FROM home_blog;

"""

cursor.execute(query)
rows = cursor.fetchall()

for i in rows:
    print(i)

(1, 'rings', 'rings of lords', 'c.zhzkhdh.vx.dkbvxvxv', datetime.datetime(2024, 1, 11, 19, 1, 3, 266020, tzinfo=datetime.timezone.utc), datetime.time(19, 1), 1)


# **Ollama**

In [12]:
def call_llama(model, prompt, stream=False):
    url = "http://localhost:11434/api/generate"
    data = {
        "model": model,
        "prompt":prompt,
        "stream":stream
    }
    
    json_data = json.dumps(data)
    response = requests.post(url, data=json_data, headers={"Content-Type":"application/json"})
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: Status code is: {response.status_code}"

In [11]:
call_llama('llama2', "hi")['response']

"Hello! It's nice to meet you. How are you today?"

In [13]:
text_idea_ai = call_llama('llama2', "Please suggest 2 topics in the field of machine learning for me.")['response']

'\n Of course! Here are five potential topics related to machine learning that you may find interesting:\n\n1. Natural Language Processing (NLP): This is a subfield of machine learning that focuses on developing algorithms and models that can process, analyze, and generate human-like language. Some potential topics in this area include text classification, sentiment analysis, machine translation, and chatbots.\n2. Computer Vision: Machine learning algorithms are increasingly being used to analyze and understand visual data from images and videos. Topics in this area could include object detection, image segmentation, facial recognition, and image generation.\n3. Reinforcement Learning: This is a type of machine learning that involves training agents to make decisions based on rewards or penalties. Potential topics in this area include deep reinforcement learning, multi-agent reinforcement learning, and applications of reinforcement learning in robotics and autonomous vehicles.\n4. Time

In [ ]:
title
abstract
content
date_posted
author
reading_time